# Question 1: Modeling Team Skill and Competition Structure

In this notebook, we model team strength as a latent variable. Each team $i$ has:
- $\mu_i$ = mean (latent) skill
- $\sigma_i$ = game-to-game variability

When team $i$ plays team $j$, we simulate a single-game "performance" value for each team:
$$X_i \sim \mathcal{N}(\mu_i, \sigma_i), \quad X_j \sim \mathcal{N}(\mu_j, \sigma_j), \quad \text{winner} = \arg\max\{X_i, X_j\}$$

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

# Set random seed for reproducibility
np.random.seed(42)

---
## (a) Choose and Visualize a Skill Distribution

**"Assume each team's mean skill was drawn from a Normal distribution."**

### Mathematical Definition
$$\mu_i \sim \mathcal{N}(\mu_0, \tau)$$

where:
- $\mu_0 = 100$ is the league-average skill level (centering parameter)
- $\tau = 15$ is the standard deviation of skill across teams

### Parameter Interpretation
- **$\mu_0 = 100$**: This represents the average skill level in the league. We use 100 as a convenient baseline (similar to IQ scaling).
- **$\tau = 15$**: This captures how spread out team skills are. With $\tau = 15$, about 68% of teams have skills between 85 and 115, and about 95% between 70 and 130.

### Why This Distribution is Plausible
A Normal distribution for team skills is plausible because:
1. **Central Limit Theorem**: Team strength aggregates many factors (player talent, coaching, resources, chemistry), and the sum of many independent factors tends toward normality.
2. **Empirical Evidence**: In most sports leagues, there's a "middle of the pack" with a few exceptional and a few poor teams - consistent with a bell curve.
3. **Continuous Skill**: Unlike discrete categories, team skill exists on a continuum, which normal distributions model well.
4. **Symmetric Spread**: It's reasonable to assume teams are equally likely to be above or below average.

In [ ]:
# Parameters for skill distribution
mu_0 = 100  # League average skill
tau = 15    # Standard deviation of skill across teams

# Create x values for PDF plot
x = np.linspace(mu_0 - 4*tau, mu_0 + 4*tau, 1000)
pdf = stats.norm.pdf(x, mu_0, tau)

# Also generate samples for histogram
samples = np.random.normal(mu_0, tau, 10000)

# Create figure with two subplots
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: PDF curve
axes[0].plot(x, pdf, 'b-', linewidth=2, label=f'$\mathcal{{N}}({mu_0}, {tau})$')
axes[0].fill_between(x, pdf, alpha=0.3)
axes[0].axvline(mu_0, color='red', linestyle='--', label=f'Mean = {mu_0}')
axes[0].axvline(mu_0 - tau, color='green', linestyle=':', alpha=0.7, label=f'±1σ = [{mu_0-tau}, {mu_0+tau}]')
axes[0].axvline(mu_0 + tau, color='green', linestyle=':', alpha=0.7)
axes[0].set_xlabel('Team Skill (μ)', fontsize=12)
axes[0].set_ylabel('Probability Density', fontsize=12)
axes[0].set_title('PDF of Team Skill Distribution', fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Histogram of samples
axes[1].hist(samples, bins=50, density=True, alpha=0.7, color='steelblue', edgecolor='black')
axes[1].plot(x, pdf, 'r-', linewidth=2, label='Theoretical PDF')
axes[1].set_xlabel('Team Skill (μ)', fontsize=12)
axes[1].set_ylabel('Density', fontsize=12)
axes[1].set_title('Histogram of 10,000 Sampled Team Skills', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('part_a_skill_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Skill Distribution: μ_i ~ N({mu_0}, {tau})")
print(f"68% of teams have skill between {mu_0-tau} and {mu_0+tau}")
print(f"95% of teams have skill between {mu_0-2*tau} and {mu_0+2*tau}")

**Figure Caption**: The left panel shows the probability density function (PDF) of the Normal distribution $\mathcal{N}(100, 15)$ used to model team skills. The right panel shows a histogram of 10,000 samples drawn from this distribution, confirming it matches the theoretical PDF. The vertical lines indicate the mean and one standard deviation bounds.

---
## (b) Create a League with N = 64 Teams

We create 64 teams by:
1. Sampling each team's mean skill $\mu_i$ from $\mathcal{N}(100, 15)$
2. Assigning each team a game-to-game variability $\sigma_i$

For $\sigma_i$, we use a uniform distribution: $\sigma_i \sim \text{Uniform}(5, 15)$. This reflects that some teams are more consistent (lower σ) while others are more variable (higher σ).

In [ ]:
# Set seed for reproducibility
rng = np.random.default_rng(42)

N = 64  # Number of teams

# Sample mean skills from N(100, 15)
mu_values = rng.normal(mu_0, tau, N)

# Sample variabilities from Uniform(5, 15)
sigma_values = rng.uniform(5, 15, N)

# Create DataFrame
teams_df = pd.DataFrame({
    'team_id': range(1, N + 1),
    'mu': mu_values,
    'sigma': sigma_values
})

print("League DataFrame (first 10 teams):")
print(teams_df.head(10).to_string(index=False))
print(f"\nTotal teams: {len(teams_df)}")
print(f"\nSkill (μ) statistics:")
print(f"  Mean: {teams_df['mu'].mean():.2f}")
print(f"  Std:  {teams_df['mu'].std():.2f}")
print(f"  Min:  {teams_df['mu'].min():.2f}")
print(f"  Max:  {teams_df['mu'].max():.2f}")
print(f"\nVariability (σ) statistics:")
print(f"  Mean: {teams_df['sigma'].mean():.2f}")
print(f"  Min:  {teams_df['sigma'].min():.2f}")
print(f"  Max:  {teams_df['sigma'].max():.2f}")

In [ ]:
# Plot teams on a 1D skill axis (number line) with error bars for sigma
fig, ax = plt.subplots(figsize=(14, 4))

# Sort by mu for better visualization
teams_sorted = teams_df.sort_values('mu').reset_index(drop=True)

# Scatter plot with y=0 for all points
y_positions = np.zeros(N)

# Create scatter plot with error bars
colors = plt.cm.coolwarm((teams_sorted['mu'] - teams_sorted['mu'].min()) / 
                          (teams_sorted['mu'].max() - teams_sorted['mu'].min()))

ax.errorbar(teams_sorted['mu'], y_positions, 
            xerr=teams_sorted['sigma'], 
            fmt='none', ecolor='gray', alpha=0.5, capsize=2)

scatter = ax.scatter(teams_sorted['mu'], y_positions, 
                     c=teams_sorted['mu'], cmap='coolwarm', 
                     s=100, edgecolors='black', linewidths=0.5, zorder=5)

# Formatting
ax.set_xlabel('Team Skill (μ)', fontsize=12)
ax.set_title('64 Teams on a 1D Skill Axis (with σ error bars)', fontsize=14)
ax.set_yticks([])
ax.set_ylim(-0.5, 0.5)
ax.axhline(0, color='black', linewidth=0.5)
ax.grid(True, axis='x', alpha=0.3)

# Add colorbar
cbar = plt.colorbar(scatter, ax=ax, orientation='horizontal', pad=0.2, aspect=40)
cbar.set_label('Team Skill (μ)', fontsize=10)

plt.tight_layout()
plt.savefig('part_b_team_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

**Figure Caption**: The 64 teams plotted on a 1D skill axis. Each point represents a team's mean skill (μ), and horizontal error bars show their game-to-game variability (σ). Colors indicate skill level (blue = lower, red = higher).

### Description of the Distribution

**Clustering**: The teams are roughly evenly distributed across the skill range, consistent with the normal distribution used to generate skills. There's a natural clustering around the mean (≈100), with the density tapering off at the extremes.

**Outliers**: There are a few teams at both extremes - the weakest teams (μ < 75) and the strongest teams (μ > 125) could be considered outliers, being more than 1.5 standard deviations from the mean.

**Variability vs. Skill Differences**: 
- The typical variability σ ranges from 5 to 15 (mean ≈ 10)
- The typical skill difference between adjacent teams (when sorted) is about 1-3 points
- The full range of skills spans about 50-60 points (from lowest to highest μ)
- Since σ can be as large as 15 and typical skill gaps are small, there's significant opportunity for upsets. A team with μ=90 and σ=10 could easily beat a team with μ=100 and σ=10 on any given day.

---
## (c) Simulate a Single Game

We implement a function `play_game` that simulates one game between two teams.

In [ ]:
def play_game(teamA, teamB, rng, return_performances=False):
    """
    Simulate a single game between two teams.
    
    Parameters:
    -----------
    teamA : dict or pd.Series
        Must contain 'team_id', 'mu', and 'sigma'
    teamB : dict or pd.Series
        Must contain 'team_id', 'mu', and 'sigma'
    rng : np.random.Generator
        Random number generator
    return_performances : bool
        If True, also return the sampled performance values
    
    Returns:
    --------
    winner : dict/Series
        The winning team
    (optional) performances : tuple
        (perf_A, perf_B) - the sampled performance values
    """
    # Sample performance values from Normal distributions
    perf_A = rng.normal(teamA['mu'], teamA['sigma'])
    perf_B = rng.normal(teamB['mu'], teamB['sigma'])
    
    # Determine winner (higher performance wins)
    if perf_A > perf_B:
        winner = teamA
    else:
        winner = teamB
    
    if return_performances:
        return winner, (perf_A, perf_B)
    return winner

In [ ]:
# Demonstrate the function by running the same matchup multiple times
rng = np.random.default_rng(123)

# Select two teams with different skills
teamA = teams_df.iloc[0]  # First team
teamB = teams_df.iloc[1]  # Second team

print(f"Matchup: Team {int(teamA['team_id'])} vs Team {int(teamB['team_id'])}")
print(f"Team {int(teamA['team_id'])}: μ = {teamA['mu']:.2f}, σ = {teamA['sigma']:.2f}")
print(f"Team {int(teamB['team_id'])}: μ = {teamB['mu']:.2f}, σ = {teamB['sigma']:.2f}")
print("\nRunning 10 games:")
print("-" * 60)

wins_A = 0
wins_B = 0

for i in range(10):
    winner, (perf_A, perf_B) = play_game(teamA, teamB, rng, return_performances=True)
    winner_id = int(winner['team_id'])
    if winner_id == teamA['team_id']:
        wins_A += 1
    else:
        wins_B += 1
    print(f"Game {i+1}: Perf_A = {perf_A:7.2f}, Perf_B = {perf_B:7.2f} -> Winner: Team {winner_id}")

print("-" * 60)
print(f"Results: Team {int(teamA['team_id'])} won {wins_A} times, Team {int(teamB['team_id'])} won {wins_B} times")
print("\nThis demonstrates that outcomes can differ even with the same matchup!")

In [ ]:
# Run many simulations to show win probability
rng = np.random.default_rng(456)
n_simulations = 10000

# Pick a clear favorite and underdog
favorite = teams_df.loc[teams_df['mu'].idxmax()]  # Highest skill
underdog = teams_df.loc[teams_df['mu'].idxmin()]  # Lowest skill

favorite_wins = 0
for _ in range(n_simulations):
    winner = play_game(favorite, underdog, rng)
    if winner['team_id'] == favorite['team_id']:
        favorite_wins += 1

print(f"\nMatchup: Best Team vs Worst Team ({n_simulations} simulations)")
print(f"Favorite (Team {int(favorite['team_id'])}): μ = {favorite['mu']:.2f}, σ = {favorite['sigma']:.2f}")
print(f"Underdog (Team {int(underdog['team_id'])}): μ = {underdog['mu']:.2f}, σ = {underdog['sigma']:.2f}")
print(f"\nFavorite win rate: {favorite_wins/n_simulations*100:.1f}%")
print(f"Upset rate: {(n_simulations-favorite_wins)/n_simulations*100:.1f}%")

---
## (d) 64-Team Single-Elimination Tournament

We implement a function to run a complete 64-team knockout tournament with match logging.

In [ ]:
def run_knockout(teams, rng, seeding='skill'):
    """
    Run a 64-team single-elimination tournament.
    
    Parameters:
    -----------
    teams : pd.DataFrame
        DataFrame with columns: team_id, mu, sigma
    rng : np.random.Generator
        Random number generator
    seeding : str
        'skill' - sort by mu (highest first), pair 1v64, 2v63, etc.
        'random' - shuffle teams randomly
    
    Returns:
    --------
    champion : pd.Series
        The winning team
    match_log : pd.DataFrame
        Log of all matches with columns: round_number, teamA_id, teamB_id, winner_id
    """
    # Create match log
    match_log = []
    
    # Set up initial bracket based on seeding
    if seeding == 'skill':
        # Sort by mu (descending) and create seeded pairings
        sorted_teams = teams.sort_values('mu', ascending=False).reset_index(drop=True)
        # Pair: 1 vs 64, 2 vs 63, ..., 32 vs 33
        n = len(sorted_teams)
        bracket = []
        for i in range(n // 2):
            bracket.append(sorted_teams.iloc[i])       # Seed i+1
            bracket.append(sorted_teams.iloc[n-1-i])   # Seed n-i
    elif seeding == 'random':
        # Randomly shuffle teams
        shuffled_idx = rng.permutation(len(teams))
        bracket = [teams.iloc[i] for i in shuffled_idx]
    else:
        raise ValueError(f"Unknown seeding method: {seeding}")
    
    # Run tournament rounds
    current_teams = bracket
    round_number = 1
    round_names = {1: 'Round of 64', 2: 'Round of 32', 3: 'Sweet 16', 
                   4: 'Elite 8', 5: 'Final 4', 6: 'Championship'}
    
    while len(current_teams) > 1:
        next_round = []
        
        # Pair up teams and play matches
        for i in range(0, len(current_teams), 2):
            teamA = current_teams[i]
            teamB = current_teams[i + 1]
            
            winner = play_game(teamA, teamB, rng)
            
            # Log the match
            match_log.append({
                'round_number': round_number,
                'round_name': round_names.get(round_number, f'Round {round_number}'),
                'teamA_id': int(teamA['team_id']),
                'teamB_id': int(teamB['team_id']),
                'winner_id': int(winner['team_id'])
            })
            
            next_round.append(winner)
        
        current_teams = next_round
        round_number += 1
    
    champion = current_teams[0]
    match_log_df = pd.DataFrame(match_log)
    
    return champion, match_log_df

### Seeding Approach

I implemented two seeding methods:

1. **Skill-based seeding (`seeding='skill'`)**: Teams are sorted by their mean skill μ in descending order. The highest-ranked team (seed 1) plays the lowest-ranked team (seed 64), seed 2 plays seed 63, and so on. This is similar to how real tournaments like March Madness work, where the bracket is designed so that the best teams don't meet until later rounds.

2. **Random seeding (`seeding='random'`)**: Teams are randomly shuffled into the bracket with no regard for skill. This means strong teams might face each other early.

---
## (e) Run One Tournament and Summarize

In [ ]:
# Run one tournament with skill-based seeding
rng = np.random.default_rng(42)
champion, match_log = run_knockout(teams_df, rng, seeding='skill')

print("=" * 60)
print("TOURNAMENT RESULTS (Skill-Based Seeding)")
print("=" * 60)

print(f"\n🏆 CHAMPION: Team {int(champion['team_id'])}")
print(f"   Mean Skill (μ): {champion['mu']:.2f}")
print(f"   Variability (σ): {champion['sigma']:.2f}")

# Find the champion's seed
sorted_by_skill = teams_df.sort_values('mu', ascending=False).reset_index(drop=True)
seed = sorted_by_skill[sorted_by_skill['team_id'] == champion['team_id']].index[0] + 1
print(f"   Seed: {seed}")

print("\n" + "-" * 60)
print("MATCH LOG SUMMARY")
print("-" * 60)

for round_num in match_log['round_number'].unique():
    round_matches = match_log[match_log['round_number'] == round_num]
    round_name = round_matches['round_name'].iloc[0]
    print(f"\n{round_name} ({len(round_matches)} matches):")
    for _, match in round_matches.iterrows():
        print(f"  Team {match['teamA_id']:2d} vs Team {match['teamB_id']:2d} -> Winner: Team {match['winner_id']}")

In [ ]:
# Show match log DataFrame
print("\nComplete Match Log DataFrame:")
print(match_log.to_string(index=False))

---
## (f) Run Tournament Many Times (μ-Based Seeding)

We run 500 tournaments to analyze championship probabilities.

In [ ]:
# Run many tournaments with skill-based seeding
n_tournaments = 500
rng = np.random.default_rng(12345)

# Track championship counts
championship_counts_skill = {int(team_id): 0 for team_id in teams_df['team_id']}

print(f"Running {n_tournaments} tournaments with skill-based seeding...")
for i in range(n_tournaments):
    champion, _ = run_knockout(teams_df, rng, seeding='skill')
    championship_counts_skill[int(champion['team_id'])] += 1
    if (i + 1) % 100 == 0:
        print(f"  Completed {i + 1} tournaments")

print("Done!")

In [ ]:
# Identify the highest-μ team
highest_mu_team = teams_df.loc[teams_df['mu'].idxmax()]
highest_mu_id = int(highest_mu_team['team_id'])
highest_mu_wins = championship_counts_skill[highest_mu_id]

print(f"\nHighest-μ team: Team {highest_mu_id}")
print(f"  μ = {highest_mu_team['mu']:.2f}, σ = {highest_mu_team['sigma']:.2f}")
print(f"  Championships: {highest_mu_wins} out of {n_tournaments}")
print(f"  Win proportion: {highest_mu_wins/n_tournaments*100:.1f}%")

In [ ]:
# Create DataFrame with championship probabilities and true skill ranks
teams_analysis = teams_df.copy()
teams_analysis['championships'] = teams_analysis['team_id'].apply(lambda x: championship_counts_skill[int(x)])
teams_analysis['championship_prob'] = teams_analysis['championships'] / n_tournaments

# Add true skill rank (1 = highest μ)
teams_analysis['true_skill_rank'] = teams_analysis['mu'].rank(ascending=False).astype(int)

# Sort by true skill rank for plotting
teams_analysis = teams_analysis.sort_values('true_skill_rank')

In [ ]:
# Plot championship probability vs true skill rank
fig, ax = plt.subplots(figsize=(12, 6))

ax.bar(teams_analysis['true_skill_rank'], teams_analysis['championship_prob'], 
       color='steelblue', edgecolor='black', alpha=0.7)

ax.set_xlabel('True Skill Rank (1 = Best)', fontsize=12)
ax.set_ylabel('Championship Probability', fontsize=12)
ax.set_title(f'Championship Probability vs True Skill Rank\n(Skill-Based Seeding, {n_tournaments} tournaments)', fontsize=14)
ax.grid(True, axis='y', alpha=0.3)

# Add horizontal line for expected value if all teams were equal
ax.axhline(1/64, color='red', linestyle='--', label=f'Random baseline (1/64 = {1/64:.3f})')
ax.legend()

plt.tight_layout()
plt.savefig('part_f_championship_prob_skill_seeding.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Top 5 teams by championship probability
top5 = teams_analysis.nlargest(5, 'championship_prob')[['team_id', 'mu', 'sigma', 'true_skill_rank', 'championships', 'championship_prob']]

print("\nTop 5 Teams by Championship Probability (Skill-Based Seeding):")
print("=" * 80)
print(f"{'Team ID':<10} {'μ':<10} {'σ':<10} {'Skill Rank':<12} {'Titles':<10} {'Win Prob':<10}")
print("-" * 80)
for _, row in top5.iterrows():
    print(f"{int(row['team_id']):<10} {row['mu']:<10.2f} {row['sigma']:<10.2f} {int(row['true_skill_rank']):<12} {int(row['championships']):<10} {row['championship_prob']:<10.3f}")

---
## (g) Re-run Under Random Seeding and Compare

In [ ]:
# Run many tournaments with random seeding
rng = np.random.default_rng(12345)  # Same seed for fair comparison

# Track championship counts
championship_counts_random = {int(team_id): 0 for team_id in teams_df['team_id']}

print(f"Running {n_tournaments} tournaments with random seeding...")
for i in range(n_tournaments):
    champion, _ = run_knockout(teams_df, rng, seeding='random')
    championship_counts_random[int(champion['team_id'])] += 1
    if (i + 1) % 100 == 0:
        print(f"  Completed {i + 1} tournaments")

print("Done!")

In [ ]:
# Add random seeding results to analysis DataFrame
teams_analysis['championships_random'] = teams_analysis['team_id'].apply(lambda x: championship_counts_random[int(x)])
teams_analysis['championship_prob_random'] = teams_analysis['championships_random'] / n_tournaments

# Highest-μ team results under random seeding
highest_mu_wins_random = championship_counts_random[highest_mu_id]

print(f"\nHighest-μ team (Team {highest_mu_id}) results:")
print(f"  Skill-based seeding: {highest_mu_wins}/{n_tournaments} = {highest_mu_wins/n_tournaments*100:.1f}%")
print(f"  Random seeding: {highest_mu_wins_random}/{n_tournaments} = {highest_mu_wins_random/n_tournaments*100:.1f}%")

In [ ]:
# Comparison plot
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Plot 1: Championship probability vs rank for both seeding methods
width = 0.4
x = np.arange(len(teams_analysis))

# Sort by true skill rank
teams_sorted = teams_analysis.sort_values('true_skill_rank')

axes[0].bar(x - width/2, teams_sorted['championship_prob'], width, 
            label='Skill-Based Seeding', color='steelblue', alpha=0.7)
axes[0].bar(x + width/2, teams_sorted['championship_prob_random'], width, 
            label='Random Seeding', color='coral', alpha=0.7)

axes[0].set_xlabel('True Skill Rank (1 = Best)', fontsize=12)
axes[0].set_ylabel('Championship Probability', fontsize=12)
axes[0].set_title('Championship Probability by Seeding Method', fontsize=14)
axes[0].set_xticks(x[::4])  # Show every 4th tick
axes[0].set_xticklabels(teams_sorted['true_skill_rank'].values[::4])
axes[0].axhline(1/64, color='gray', linestyle='--', alpha=0.5, label='Random baseline')
axes[0].legend()
axes[0].grid(True, axis='y', alpha=0.3)

# Plot 2: Scatter plot comparing probabilities
axes[1].scatter(teams_analysis['championship_prob'], 
                teams_analysis['championship_prob_random'],
                c=teams_analysis['true_skill_rank'], cmap='coolwarm_r',
                s=80, alpha=0.7, edgecolors='black', linewidths=0.5)

# Add diagonal line
max_prob = max(teams_analysis['championship_prob'].max(), 
               teams_analysis['championship_prob_random'].max())
axes[1].plot([0, max_prob], [0, max_prob], 'k--', alpha=0.5, label='Equal probability')

axes[1].set_xlabel('Championship Prob (Skill-Based Seeding)', fontsize=12)
axes[1].set_ylabel('Championship Prob (Random Seeding)', fontsize=12)
axes[1].set_title('Seeding Method Comparison', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

cbar = plt.colorbar(axes[1].collections[0], ax=axes[1])
cbar.set_label('True Skill Rank (1=Best)', fontsize=10)

plt.tight_layout()
plt.savefig('part_g_seeding_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Top 5 teams by championship probability under random seeding
top5_random = teams_analysis.nlargest(5, 'championship_prob_random')[['team_id', 'mu', 'sigma', 'true_skill_rank', 'championships_random', 'championship_prob_random']]

print("\nTop 5 Teams by Championship Probability (Random Seeding):")
print("=" * 80)
print(f"{'Team ID':<10} {'μ':<10} {'σ':<10} {'Skill Rank':<12} {'Titles':<10} {'Win Prob':<10}")
print("-" * 80)
for _, row in top5_random.iterrows():
    print(f"{int(row['team_id']):<10} {row['mu']:<10.2f} {row['sigma']:<10.2f} {int(row['true_skill_rank']):<12} {int(row['championships_random']):<10} {row['championship_prob_random']:<10.3f}")

In [ ]:
# Summary statistics
print("\n" + "=" * 70)
print("SUMMARY: THE ROLE OF SEEDING IN SINGLE-ELIMINATION TOURNAMENTS")
print("=" * 70)

# Calculate some key metrics
top_10_skill = teams_analysis.nsmallest(10, 'true_skill_rank')

top10_prob_skill = top_10_skill['championship_prob'].sum()
top10_prob_random = top_10_skill['championship_prob_random'].sum()

print(f"\n1. Probability that a top-10 (by μ) team wins:")
print(f"   Skill-based seeding: {top10_prob_skill*100:.1f}%")
print(f"   Random seeding: {top10_prob_random*100:.1f}%")

# Gini coefficient or concentration measure
print(f"\n2. Best team's (highest μ) championship probability:")
print(f"   Skill-based seeding: {teams_analysis.loc[teams_analysis['true_skill_rank']==1, 'championship_prob'].values[0]*100:.1f}%")
print(f"   Random seeding: {teams_analysis.loc[teams_analysis['true_skill_rank']==1, 'championship_prob_random'].values[0]*100:.1f}%")

# Number of unique champions
unique_champions_skill = sum(1 for c in championship_counts_skill.values() if c > 0)
unique_champions_random = sum(1 for c in championship_counts_random.values() if c > 0)

print(f"\n3. Number of unique champions (out of 64 teams):")
print(f"   Skill-based seeding: {unique_champions_skill}")
print(f"   Random seeding: {unique_champions_random}")

### Analysis: The Role of Seeding in Single-Elimination Tournaments

**Key Findings:**

1. **Skill-based seeding HELPS top teams**: Under skill-based seeding, the best teams have a higher probability of winning the tournament. This is because they face weaker opponents in early rounds, saving their toughest matchups for later when fewer games need to be won.

2. **Random seeding creates more parity**: With random seeding, the best teams can be knocked out early by facing each other. This reduces their championship probability and spreads wins across more teams.

3. **Championship probability is more concentrated with seeding**: The scatter plot shows that top teams (blue points) are above the diagonal, meaning they have higher championship probability under skill-based seeding. Lower-ranked teams (red points) are below the diagonal.

4. **Seeding protects favorites but doesn't guarantee success**: Even with favorable seeding, the best team typically wins less than 20-30% of tournaments due to game-to-game variability. Single-elimination tournaments inherently have high variance because one bad game ends your run.

**Why does seeding matter?**

- In a single-elimination tournament, you must win 6 consecutive games to be champion
- With seeding, the #1 seed's expected opponent quality starts low and gradually increases
- Without seeding, the #1 seed might face the #2 seed in Round 1, with the winner exhausting luck early
- Seeding ensures that if the best teams both play to their ability, they meet in the final (maximizing drama and "deserved" outcomes)

**Implications for real sports:**
This explains why leagues use seeding based on regular season performance (like March Madness or NFL playoffs). It rewards regular-season success and makes it more likely that the "best" team wins, though upsets remain possible due to single-game variance.

---
---
# Question 2: Tournament Structure as a Network

Tournament structure is not just "rules"—it determines who plays whom and therefore how much information we get about underlying skill. In this question we represent a tournament as a network and compare structures.

---
## (a) Represent Knockout Tournament as a NetworkX Graph

Using the match log from Question 1, we build a graph representation of one simulated tournament and visualize it using `networkx`.

In [ ]:
import networkx as nx

# Re-run a tournament to get a fresh match log (using the same teams from Q1)
rng = np.random.default_rng(42)
champion, match_log = run_knockout(teams_df, rng, seeding='skill')

print("Match log for the tournament:")
print(match_log.to_string(index=False))

In [ ]:
def build_tournament_graph(match_log):
    """
    Build a directed graph from a tournament match log.
    
    Each node represents a team.
    Each directed edge (A -> B) means team A lost to team B (B defeated A).
    Edge attributes include round number.
    
    Parameters:
    -----------
    match_log : pd.DataFrame
        DataFrame with columns: round_number, teamA_id, teamB_id, winner_id
    
    Returns:
    --------
    G : nx.DiGraph
        Directed graph representing the tournament
    """
    G = nx.DiGraph()
    
    # Add all teams as nodes
    all_teams = set(match_log['teamA_id']).union(set(match_log['teamB_id']))
    G.add_nodes_from(all_teams)
    
    # Add edges: loser -> winner (showing who beat whom)
    for _, match in match_log.iterrows():
        winner = match['winner_id']
        teamA = match['teamA_id']
        teamB = match['teamB_id']
        loser = teamA if winner == teamB else teamB
        
        # Edge from loser to winner
        G.add_edge(loser, winner, round=match['round_number'], round_name=match['round_name'])
    
    return G

# Build the graph
G = build_tournament_graph(match_log)

print(f"Graph has {G.number_of_nodes()} nodes (teams)")
print(f"Graph has {G.number_of_edges()} edges (matches)")
print(f"\nChampion (node with no outgoing edges indicating a loss): Team {champion['team_id']:.0f}")

In [ ]:
# Visualize the tournament graph
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# ---- Plot 1: Simple circular layout ----
ax1 = axes[0]
pos_circular = nx.circular_layout(G)

# Color nodes by how far they got (in-degree = number of wins)
wins = dict(G.in_degree())
node_colors = [wins[node] for node in G.nodes()]

nx.draw(G, pos_circular, ax=ax1,
        with_labels=True,
        node_color=node_colors,
        cmap=plt.cm.YlOrRd,
        node_size=500,
        font_size=8,
        font_weight='bold',
        arrows=True,
        arrowsize=10,
        edge_color='gray',
        alpha=0.8)

ax1.set_title('Tournament Graph (Circular Layout)\nEdge: Loser → Winner, Color: # of Wins', fontsize=12)

# ---- Plot 2: Hierarchical layout showing bracket structure ----
ax2 = axes[1]

# Create a custom hierarchical layout based on round eliminated
def get_tournament_positions(match_log, teams_df):
    """
    Create positions for a bracket-style visualization.
    x = round eliminated (or 7 for champion)
    y = spread teams vertically within each round
    """
    pos = {}
    
    # Find when each team was eliminated
    eliminated_round = {}
    for _, match in match_log.iterrows():
        winner = match['winner_id']
        teamA = match['teamA_id']
        teamB = match['teamB_id']
        loser = teamA if winner == teamB else teamB
        eliminated_round[loser] = match['round_number']
    
    # Champion was never eliminated
    all_teams = set(match_log['teamA_id']).union(set(match_log['teamB_id']))
    champ = [t for t in all_teams if t not in eliminated_round][0]
    eliminated_round[champ] = 7  # Place champion at the end
    
    # Group teams by elimination round
    round_teams = {}
    for team, rd in eliminated_round.items():
        if rd not in round_teams:
            round_teams[rd] = []
        round_teams[rd].append(team)
    
    # Assign positions
    for rd, teams in round_teams.items():
        n_teams = len(teams)
        for i, team in enumerate(sorted(teams)):
            x = rd
            y = (i - n_teams/2) * (8 / max(n_teams, 1))  # Spread vertically
            pos[team] = (x, y)
    
    return pos

pos_bracket = get_tournament_positions(match_log, teams_df)

nx.draw(G, pos_bracket, ax=ax2,
        with_labels=True,
        node_color=node_colors,
        cmap=plt.cm.YlOrRd,
        node_size=400,
        font_size=7,
        font_weight='bold',
        arrows=True,
        arrowsize=8,
        edge_color='gray',
        alpha=0.8)

# Add round labels
round_labels = {1: 'R64', 2: 'R32', 3: 'S16', 4: 'E8', 5: 'F4', 6: 'Final', 7: 'Champ'}
for rd, label in round_labels.items():
    ax2.text(rd, -20, label, ha='center', fontsize=10, fontweight='bold')

ax2.set_title('Tournament Bracket Structure\n(x = round eliminated, Champion at right)', fontsize=12)
ax2.set_xlim(0, 8)

plt.tight_layout()
plt.savefig('question2a_tournament_graph.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n🏆 Champion: Team {int(champion['team_id'])} (shown at the far right with most wins)")

In [ ]:
# Additional graph analysis
print("Graph Properties:")
print(f"  - Number of nodes (teams): {G.number_of_nodes()}")
print(f"  - Number of edges (matches): {G.number_of_edges()}")
print(f"  - Is it a DAG (directed acyclic graph)? {nx.is_directed_acyclic_graph(G)}")

# The champion has in-degree = 6 (won 6 matches) and out-degree = 0 (never lost)
print(f"\nChampion's degree:")
print(f"  - In-degree (wins): {G.in_degree(int(champion['team_id']))}")
print(f"  - Out-degree (losses): {G.out_degree(int(champion['team_id']))}")

# Teams eliminated in Round 1 have in-degree = 0, out-degree = 1
round1_losers = [node for node in G.nodes() if G.in_degree(node) == 0 and G.out_degree(node) == 1]
print(f"\nTeams eliminated in Round 1: {len(round1_losers)} teams")

**Figure Description**: The left panel shows the tournament as a directed graph with circular layout. Each node is a team, and edges point from loser to winner. Node color indicates the number of wins (darker = more wins). The right panel shows a bracket-style layout where the x-axis represents the round in which a team was eliminated, with the champion at the far right.

---
## (b) Alternative Competition Structure: Swiss System Tournament

### Description of the Rules

The **Swiss System** is a non-eliminating tournament format that provides a middle ground between round-robin (everyone plays everyone) and single-elimination (one loss and you're out). The key rules are:

1. **Fixed number of rounds**: The tournament runs for a predetermined number of rounds (typically $\lceil \log_2(N) \rceil$ to $\lceil \log_2(N) \rceil + 2$ rounds for $N$ participants).

2. **No elimination**: All players/teams continue to play in every round, regardless of previous results.

3. **Pairing by score**: In each round after the first, players are paired with opponents who have the same (or similar) cumulative score. Winners play winners, losers play losers.

4. **No repeat matchups**: A player cannot face the same opponent twice during the tournament.

5. **Final ranking**: At the end, players are ranked by their total points (wins). Tiebreakers often include head-to-head results, strength of schedule, or Buchholz score (sum of opponents' scores).

### Real Example: FIDE Chess Tournaments

The **Chess Olympiad** is one of the largest team chess events, where 180+ national teams compete over 11 rounds using the Swiss System. This format allows a massive field to determine rankings without requiring the thousands of games a full round-robin would need.

**Source**: FIDE Handbook, Section C.04 - Swiss System Rules  
https://handbook.fide.com/chapter/C0403

Another example is **Magic: The Gathering Grand Prix/MagicFests**, where hundreds of players compete in Swiss rounds before a single-elimination Top 8 playoff.

### Hypothesis: Best and Worst Skill Distributions for Swiss System

**Best suited for:**
- **Large field with dispersed skills**: When you have many participants (100+) with a wide range of skill levels, Swiss efficiently separates the top performers without requiring $O(N^2)$ games like round-robin.
- **Moderate variance ($\sigma$)**: With moderate game-to-game variability, the Swiss system's multiple rounds help the cream rise to the top while allowing recovery from a single bad game.
- **Clear skill tiers**: When teams cluster into distinct skill tiers, Swiss quickly sorts them as similar-record teams play each other.

**Worst suited for:**
- **Very similar $\mu$ values (compressed skill distribution)**: When many teams have nearly identical skill, the Swiss system struggles because early-round pairings are essentially random among equals, and outcomes depend heavily on who you happen to face. More rounds are needed to differentiate, but you can't repeat opponents.
- **Very high variance ($\sigma$)**: With large game-to-game variability, even the best team might lose several games to weaker opponents by chance. Since there's no elimination, a high-$\sigma$ favorite might accumulate losses and end up with mediocre standings despite high $\mu$.
- **Small number of participants**: With few teams (< 16), round-robin becomes feasible and provides more complete information about relative skill.

**Key insight**: The Swiss system trades off information completeness for efficiency. It generates $N \times R$ games (where $R$ is rounds) compared to $N(N-1)/2$ for round-robin, but provides less certainty about true rankings. This tradeoff is favorable when $N$ is large and skill differences are substantial enough to emerge in limited rounds.